In [102]:
import qiskit as qk
import math
import numpy as np

In [103]:
np.random.rand(1,2)

array([[0.92141417, 0.95811443]])

In [104]:
def testR(randn,p):
    
    if randn >= p:
        return 0
    elif randn < p/3:
        return 1
    elif randn < 2*p/3:
        return 2
    elif randn < p:
        return 3

def build_pauli_mask(n,basis,p,gatetype):
    
    if gatetype == 'CX':
        if basis == 'X' or basis == 'Y':
            r = np.random.rand(6*n-1)
        else:
            r = np.random.rand(7*n-1)
    else:
        if basis == 'X' or basis == 'Y':
            r = np.random.rand(5*n-1)
        else:
            r = np.random.rand(6*n-1)
        
    for i in range(len(r)):
        r[i] = testR(r[i],p)
    
    return r

def add_pauli(qc,qr,qubit,m):
    
    if m == 0:
        pass
    elif m == 1:
        qc.x(qr[qubit])
    elif m == 2:
        qc.y(qr[qubit])
    elif m == 3:
        qc.z(qr[qubit])
    
    return

In [157]:
def charcterization_sequence(n,p,basis,fgateparams,cname='circuit0'):
    
    qr = qk.QuantumRegister(n)
    cr = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(qr,cr,name=cname)
    
    mask = build_pauli_mask(n,basis,p,fgateparams[0])
    
    j = prepareCircuitWithPaulis(qr,cr,circuit,n,basis,mask,fgateparams)
    print(j)
    
    return circuit
        
def prepareCircuitWithPaulis(qr,cr,qc,n,basis,mask,fgateparams):
    
    j=0
    add_pauli(qc,qr,j,mask[j])
    j += 1
    
    qc.h(qr[0])
    
    for i in range(n):
        add_pauli(qc,qr,i,mask[j])
        j += 1
    
    for i in range(n-1):
        qc.cx(qr[0],qr[i+1])
        add_pauli(qc,qr,0,mask[j])
        j+=1
        add_pauli(qc,qr,i+1,mask[j])
        j+=1
    
    if basis == 'X':
        for i in range(n):
            qc.h(qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
    elif basis == 'Y':
        for i in range(n):
            qc.u3(math.pi/2,-math.pi/2,math.pi/2,qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            
    if fgateparams[0] == 'U1':
        for i in range(n):
            qc.u1(-fgateparams[1],qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            qc.u1(fgateparams[1],qr[i])
    elif fgateparams[0] == 'U2':
        for i in range(n):
            qc.u2(-fgateparams[2],-fgateparams[1],qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            qc.u2(fgateparams[1],fgateparams[2],qr[i])
    elif fgateparams[0] == 'U3':
        for i in range(n):
            qc.u3(fgateparams[1],-fgateparams[3],-fgateparams[2],qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            qc.u3(fgateparams[1],fgateparams[2],fgateparams[3],qr[i])        
    elif fgateparams[0] == 'CX':     
        for i in range(n-1):
            qc.cx(qr[i],qr[i+1])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            add_pauli(qc,qr,i+1,mask[j])
            j+=1
        qc.cx(qr[n-1],qr[0])
        add_pauli(qc,qr,n-1,mask[j])
        j+=1
        add_pauli(qc,qr,0,mask[j])
        j+=1
        
    print(j)
     
    if basis == 'Z':
        for i in range(n):
            add_pauli(qc,qr,i,mask[j])
            j+=1
            qc.h(qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            qc.measure(qr[i],cr[i])
    else:
        for i in range(n):
            qc.measure(qr[i],cr[i])
            
    return j

def prepareGHZWithPaulis(qr,cr,qc,n,basis,mask,fgateparams):
    
    j=0
    add_pauli(qc,qr,j,mask[j])
    j += 1
    
    qc.h(qr[0])
    
    for i in range(n):
        add_pauli(qc,qr,i,mask[j])
        j += 1
    
    for i in range(n-1):
        qc.cx(qr[0],qr[i+1])
        add_pauli(qc,qr,0,mask[j])
        j+=1
        add_pauli(qc,qr,i+1,mask[j])
        j+=1
    
    if basis == 'X':
        for i in range(n):
            qc.h(qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
    elif basis == 'Y':
        for i in range(n):
            #qc.u3(math.pi/2,-math.pi/2,math.pi/2,qr[i])
            qc.h(qr[i])
            qc.s(qr[i])
            add_pauli(qc,qr,i,mask[j])
            j+=1
            
    return j

def ghz_sequence(n,p,basis,fgateparams,cname='circuit0'):
    
    qr = qk.QuantumRegister(n)
    cr = qk.ClassicalRegister(n)
    circuit = qk.QuantumCircuit(qr,cr,name=cname)
    
    mask = build_pauli_mask(n,basis,p,fgateparams[0])
    
    j = prepareGHZWithPaulis(qr,cr,circuit,n,basis,mask,fgateparams)
    #print(j)
    
    return circuit


In [113]:
circ = charcterization_sequence(3,0,'Y',['U1',0],cname='circuit0')
circ.draw()

In [110]:
simulator = qk.Aer.get_backend('qasm_simulator')
job = qk.execute(circ, simulator, shots=1000)
result = job.result()
counts = result.get_counts(circ)
print(counts)

{'110': 1064, '001': 1008, '111': 1032, '000': 1056, '100': 997, '101': 961, '010': 1053, '011': 1021}


In [154]:
circ = ghz_sequence(3,0,'Y',['U1',0],cname='circuit0')
circ.draw()

In [155]:
simulator = qk.Aer.get_backend('statevector_simulator')
job = qk.execute(circ, simulator, shots=1000)
result = job.result()
print(result.get_statevector())

[ 0.5+0.j  0. +0.j  0. +0.j -0.5+0.j  0. +0.j -0.5+0.j -0.5+0.j  0. +0.j]
